In [4]:
import pandas as pd
import json

df_selected = pd.read_csv("/Users/vatsal/Desktop/SimplyTag/selected_data.csv", low_memory=False)
df_selected

,Timestamp,Trace-id,HTTP Status Code,Path,User Agent
0,2024-11-29T23:00:22.7045112Z,3dff50eaf246e6cc55173db4092d5187,NaN,NaN,NaN
1,2024-11-29T23:00:22.7065968Z,3dff50eaf246e6cc55173db4092d5187,NaN,/api/v2/versions,check_http/v2.4.0 (monitoring-plugins 2.4.0)
2,2024-11-29T23:00:22.7104302Z,3dff50eaf246e6cc55173db4092d5187,NaN,NaN,NaN
3,2024-11-29T23:00:22.7112929Z,3dff50eaf246e6cc55173db4092d5187,NaN,NaN,NaN
4,2024-11-29T23:00:22.7124052Z,3dff50eaf246e6cc55173db4092d5187,NaN,NaN,NaN
...,...,...,...,...,...
2144399,2024-11-19T23:00:03.2857262Z,e633e492116f3febe619bfed47449e89,NaN,NaN,NaN
2144400,2024-11-19T23:00:03.2859194Z,e633e492116f3febe619bfed47449e89,NaN,NaN,NaN
2144401,2024-11-19T23:00:03.2861440Z,e633e492116f3febe619bfed47449e89,NaN,NaN,NaN
2144402,2024-11-19T23:00:03.2863937Z,e633e492116f3febe619bfed47449e89,NaN,NaN,NaN


In [12]:
df_with_user_agent = df_selected[df_selected['User Agent'].notnull()]
print(f"Data Frame with non-null 'User Agent': {len(df_with_user_agent)}")

#print("Rows with non-null 'User Agent':", df_selected['User Agent'].notnull().sum())

print("Different number of 'User Agent':", df_selected['User Agent'].nunique())
df_with_user_agent


Data Frame with non-null 'User Agent': 179148
Different number of 'User Agent': 188


,Timestamp,Trace-id,HTTP Status Code,Path,User Agent
1,2024-11-29T23:00:22.7065968Z,3dff50eaf246e6cc55173db4092d5187,NaN,/api/v2/versions,check_http/v2.4.0 (monitoring-plugins 2.4.0)
14,2024-11-29T23:00:36.7767705Z,a95567c1116e63d5a81970d42e6192c2,NaN,/api/healthz,curl/8.5.0
17,2024-11-29T23:00:36.5773863Z,bb52744079ab054a8189bf62d8602370,NaN,/api/v2/versions,check_http/v2.4.0 (monitoring-plugins 2.4.0)
30,2024-11-29T23:00:37.6812329Z,82001f1d8805ffd5b558d2279faa8208,NaN,/api/healthz,curl/8.5.0
43,2024-11-29T23:01:09.1300055Z,e1184bb897246f17238b68ae4c710788,NaN,/api/healthz,curl/8.5.0
...,...,...,...,...,...
2144350,2024-11-19T22:59:39.9029863Z,f54eea3ca5d617bb425919ec247c3c9a,NaN,/api/healthz,curl/8.5.0
2144358,2024-11-19T22:59:55.7568830Z,0cfdcd4505ca4f1969017713cc115bff,NaN,/api/v2/versions,check_http/v2.4.0 (monitoring-plugins 2.4.0)
2144371,2024-11-19T22:59:57.9722823Z,d60a0fa2eeef9985449391d3a3c619a8,NaN,/api/healthz,curl/8.5.0
2144379,2024-11-19T22:59:59.0713362Z,c04e0c475d8a6a9e673f00ae3bb74e41,NaN,/api/v2/versions,check_http/v2.4.0 (monitoring-plugins 2.4.0)


In [18]:
import pandas as pd
import re

# Assuming df_with_user_agent contains the cleaned log data with a 'User Agent' column
user_agent_column = 'User Agent'

# Step 1: Group and count occurrences of each user agent
user_agent_counts = df_with_user_agent[user_agent_column].value_counts().reset_index()
user_agent_counts.columns = ['User Agent', 'Count']
print("Top User Agents:\n", user_agent_counts.head())

# Step 2: Identify known bots using regex
def is_bot(user_agent):
    bot_keywords = ['bot', 'crawl', 'spider', 'scraper', 'crawler', 'libwww', 'scan', 'wget', 'python']
    pattern = '|'.join(bot_keywords)
    return bool(re.search(pattern, user_agent, re.IGNORECASE))

user_agent_counts['Is Bot'] = user_agent_counts['User Agent'].apply(is_bot)
print("Sample Bot Detection:\n", user_agent_counts[user_agent_counts['Is Bot']].head())

# Step 3: Flag uncommon user agents
threshold = 5  # Define a threshold for "rare" user agents
user_agent_counts['Is Rare'] = user_agent_counts['Count'] <= threshold

# Step 4: Combine findings and inspect suspicious user agents
suspicious_agents = user_agent_counts[(user_agent_counts['Is Bot']) | (user_agent_counts['Is Rare'])]
print("Suspicious User Agents:\n", suspicious_agents.head())

# Step 5: Save results for further analysis
suspicious_agents.to_csv('/Users/vatsal/Desktop/SimplyTag/User-Agent/suspicious_user_agents.csv', index=False)
print("Suspicious user agents saved successfully.")


Top User Agents:
                                           User Agent  Count
0                                         curl/8.5.0  36718
1       check_http/v2.4.0 (monitoring-plugins 2.4.0)  35870
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  25883
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   9984
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...   6512
Sample Bot Detection:
                                             User Agent  Count  Is Bot
135  Expanse, a Palo Alto Networks company, searche...      2    True
Suspicious User Agents:
                                             User Agent  Count  Is Bot  Is Rare
126  Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...      5   False     True
127  Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) G...      5   False     True
128  Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; ...      4   False     True
129  Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...      4   False     True
130  Mozilla/5.0 (iPhone; CPU iPhone OS

In [16]:
!pip install user-agents


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [17]:
from user_agents import parse

def parse_user_agent(user_agent):
    ua = parse(user_agent)
    return ua.is_bot, ua.browser.family, ua.os.family, ua.device.family

user_agent_counts[['Is Bot', 'Browser', 'OS', 'Device']] = user_agent_counts['User Agent'].apply(
    lambda ua: pd.Series(parse_user_agent(ua))
)


In [20]:
import pandas as pd

# Assuming df_with_user_agent is the DataFrame containing your log data

# Step 1: Group and count occurrences of each user agent
user_agent_counts = df_with_user_agent['User Agent'].value_counts().reset_index()

# Rename columns for better understanding
user_agent_counts.columns = ['User Agent', 'Count']

# Display the top 10 most common user agents
print("Top User Agents:\n", user_agent_counts.head(10))

# Optionally, you can save this grouped data to a CSV file for further analysis
user_agent_counts.to_csv('/Users/vatsal/Desktop/SimplyTag/User-Agent/grouped_user_agents.csv', index=False)
print("User agent counts saved successfully.")


Top User Agents:
                                           User Agent  Count
0                                         curl/8.5.0  36718
1       check_http/v2.4.0 (monitoring-plugins 2.4.0)  35870
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...  25883
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   9984
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...   6512
5  Mozilla/5.0 (Linux; Android 13; SM-P615 Build/...   5689
6  Mozilla/5.0 (iPhone; CPU iPhone OS 17_4 like M...   5303
7  Mozilla/5.0 (Linux; Android 14; Pixel 6a Build...   3829
8  Mozilla/5.0 (Linux; Android 14; SM-P619 Build/...   3027
9  Mozilla/5.0 (iPhone; CPU iPhone OS 17_6_1 like...   2862
User agent counts saved successfully.
